###############################################################################
# SRSWTIDivergence
###############################################################################

In [ ]:
# ---------------------------  Imports -------------------------------
from srswti_axis import SRSWTIDivergence
from utils import call_groq_llm, SAMPLE_DOCS

/home/zhreyas/miniconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-10 14:43:18.380968: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741597998.400078  905904 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741597998.405495  905904 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 14:43:18.425138: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-c

# ---------------------------  Basic Demos -------------------------


In [4]:
divergence_analyzer = SRSWTIDivergence()
docA = SAMPLE_DOCS[0]
docB = SAMPLE_DOCS[1]
docC = SAMPLE_DOCS[2]

In [5]:
# Calculate divergence between two documents
div_score_1 = divergence_analyzer.calculate_divergence(docA, docB)
print("== Divergence Example 1 ==")
print("Divergence A vs B:", div_score_1, "\n")


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches: 100%|██████████| 1/1 [00:00<00:00, 132.91it/s]

== Divergence Example 1 ==
Divergence A vs B: 0.44198785358177467 



In [6]:
div_details_2 = divergence_analyzer.calculate_divergence(docA, docC, return_components=True)
print("== Divergence Example 2 ==")
print(div_details_2, "\n")

Batches: 100%|██████████| 1/1 [00:00<00:00, 175.77it/s]

== Divergence Example 2 ==
{'divergence_score': 0.5038743775341092, 'cosine_similarity': 0.17952489852905273, 'jensen_shannon_divergence': 0.20086232009308652, 'entropy_p': 6.927951723354903, 'entropy_q': 6.894970119991518, 'text_complexity_1': 0.9192951723354903, 'text_complexity_2': 0.9894970119991517, 'cosine_weight': 0.4890345495572271, 'jsd_weight': 0.510965450442773} 



In [16]:
# Compare multiple documents
compare_res_3 = divergence_analyzer.compare_texts([docA, docB, docC], threshold=0.449)
print("== Divergence Example 3 ==")
print(compare_res_3, "\n")
print(f"Similar texts: {compare_res_3['similar_texts']}")
print(f"Divergent texts: {compare_res_3['divergent_texts']}")

Batches: 100%|██████████| 1/1 [00:00<00:00, 158.71it/s]

== Divergence Example 3 ==
{'scores': [0.44198785358177467, 0.5038743775341092], 'similar_texts': ['## Generative AI and Blockchain Integration\n\n### 1. NFT Creation and Verification\n- Generative AI creates unique digital assets that can be minted as NFTs\n- Smart contracts can verify the authenticity and provenance of AI-generated art\n\n### 2. Decentralized AI Training\n- Blockchain-based platforms enable distributed training of AI models\n- Token incentives for contributing computing resources or training data\n\n### 3. On-chain AI Models\n- AI models deployed directly on blockchain networks\n- Enables transparent, verifiable AI inference with immutable records\n\n### 4. Governance and Data Marketplaces\n- DAOs (Decentralized Autonomous Organizations) for governing AI systems\n- Decentralized data marketplaces for training AI with fair compensation to data owners\n\n### 5. Challenges and Considerations\n- Computational limitations of current blockchain infrastructure\n- Privacy co

# -------------------  RAG Demos ----------------------


In [ ]:
def rag_demo_a(user_query, docs=SAMPLE_DOCS):
    """
    1) Compare doc divergence with user query
    2) Find minimal divergence doc
    3) Summarize with Groq
    """
    if not docs:
        print("[Theorem3 RAG Demo A] No docs.")
        return

    best_doc, min_div = None, float('inf')
    for d in docs:
        score = divergence_analyzer.calculate_divergence(user_query, d)
        if score < min_div:
            min_div = score
            best_doc = d

    prompt = (
        f"User query: '{user_query}'\n"
        f"Best doc (lowest divergence={min_div}):\n{best_doc}\n"
        "Please explain how it addresses the query."
    )
    llm_response = call_groq_llm(prompt)
    print("[Theorem3 RAG Demo A] LLM Response:\n", llm_response, "\n")

def rag_demo_b(docs=SAMPLE_DOCS):
    """
    1) Compare docs pairwise, group them
    2) Summarize each group with Groq
    """
    if not docs:
        print("[Theorem3 RAG Demo B] No docs.")
        return
    res = divergence_analyzer.compare_texts(docs, threshold=0.6)
    prompt = (
        f"Compared docs with threshold=0.6. Result:\n{res}\n"
        "Give an overview of grouping."
    )
    llm_response = call_groq_llm(prompt)
    print("[Theorem3 RAG Demo B] LLM Response:\n", llm_response, "\n")



In [ ]:
rag_demo_a("I want to learn about quantum computing", SAMPLE_DOCS)

Batches: 100%|██████████| 1/1 [00:00<00:00, 136.48it/s]


[Theorem3 RAG Demo A] LLM Response:
 [Groq LLM] This document addresses the user's query "I want to learn about quantum computing" in several ways:

1. **Introduction to quantum computing**: The document starts by explaining the fundamental principles of quantum computing, such as the use of qubits, superposition, and the differences between quantum and classical computers. This provides a solid foundation for understanding the basics of quantum computing.

2. **Potential impact and applications**: The document discusses the potential impact of quantum computing across various fields, including cryptography, drug discovery, materials science, and artificial intelligence. This gives the user an idea of the potential applications and significance of quantum computing.

3. **Challenges and limitations**: The document highlights the current challenges and limitations of quantum computing, such as qubit stability, error correction, and scaling issues. This provides a realistic understanding

In [ ]:
rag_demo_b(SAMPLE_DOCS)


Batches: 100%|██████████| 1/1 [00:00<00:00, 121.17it/s]


[Theorem3 RAG Demo B] LLM Response:
 [Groq LLM] The provided result shows a grouping of texts based on their similarity, with a threshold of 0.6. The texts are categorized into two groups: 'similar_texts' and 'divergent_texts'.

**Similar Texts:**
There are two texts that are considered similar, with similarity scores of 0.44198785358177467 and 0.5038743775341092, respectively. These texts are:

1. A text discussing the integration of generative AI and blockchain, covering topics such as NFT creation, decentralized AI training, and on-chain AI models.
2. A text about quantum computing, its principles, potential impact, and current challenges, including cryptography, drug discovery, and materials science.

**Divergent Texts:**
There are no texts that are considered divergent, as the 'divergent_texts' list is empty.

In summary, the grouping result shows that the two provided texts are considered similar, despite discussing different topics (AI and blockchain vs. quantum computing). This